# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [4]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [7]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [8]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [9]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [10]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


In [57]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,0,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,1,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,2,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,3,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,4,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


## Матрица рейтингов

In [11]:
from scipy.sparse import coo_matrix, csr_matrix

In [12]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [59]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in Compressed Sparse Row format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [14]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [15]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [16]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [17]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [18]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [140]:
user_1_rated.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [25]:
user_2_rated = (user_2 != 0)
user_2_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 468 stored elements in Compressed Sparse Row format>

In [26]:
(user_1_rated).multiply(user_2_rated)

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [22]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [24]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


In [266]:
def cosine_similarity_pair_users(u, v):
    idx = np.logical_and(u.toarray(), v.toarray())

    if np.count_nonzero(idx)>2:
        
        return u.dot(v.T)[0, 0]/(np.sqrt(u[idx].dot(u[idx].T)[0, 0]) * np.sqrt(v[idx].dot(v[idx].T)[0, 0]))
    else:
        return 0

### _Решение_

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [267]:
def get_rates(ind, M, n_neighbours):
    sim_vec = []
    for i in np.arange(0, M.shape[0]):
        if i != ind:
            sim_vec.append(cosine_similarity_pair_users(M[ind], M[i]))
        else:
            sim_vec.append(0)
    
    sim_vec = np.asarray(sim_vec)

    neighbours = sim_vec.argsort()[-n_neighbours:]
    
    rates = R[neighbours[0]]*sim_vec[neighbours[0]]
    
    for i in (neighbours[1:]):
            rates = rates + R[i]*sim_vec[i]
    

    rates = rates/sim_vec[neighbours].sum()
    
    user_rated = (R[ind] != 0)
    rates[user_rated] = 0
  
    return (rates)

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [268]:
top = get_rates(20, R, 30)

C:\Users\margarita.potokina\Anaconda3\envs\coursera_ds\lib\site-packages\ipykernel_launcher.py:22: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [269]:
ar = top.toarray()

In [270]:
ar[0][5573]

2.066125730969801

In [271]:
sr = np.argsort(ar)[0][-5:]

In [272]:
sr

array([6720, 5573,  343,  306, 2614], dtype=int64)

In [273]:
ar[0][sr]

array([2.04966266, 2.06612573, 2.20000015, 2.30002146, 2.5834227 ])

In [274]:
top5 = sr

In [275]:
print(df_movies['title'].loc[df_movies['id'] == 5573])
print(df_movies['title'].loc[df_movies['id'] == 343])
print(df_movies['title'].loc[df_movies['id'] == 826])
print(df_movies['title'].loc[df_movies['id'] == 4653])
print(df_movies['title'].loc[df_movies['id'] == 306])

5606    The Lord of the Rings: The Two Towers
Name: title, dtype: object
343    Forrest Gump
Name: title, dtype: object
831    Casablanca
Name: title, dtype: object
4679    The Lord of the Rings: The Fellowship of the Ring
Name: title, dtype: object
306    The Shawshank Redemption
Name: title, dtype: object


## Генерация ответа

In [276]:
', '.join(str(i) for i in top5)

'6720, 5573, 343, 306, 2614'